In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

def create_monthly_means(input_file, output_file, variables_to_average=None):
    """
    Create monthly mean averages from daily NetCDF data
    
    Parameters:
    -----------
    input_file : str
        Path to input NetCDF file with daily data
    output_file : str
        Path for output NetCDF file with monthly means
    variables_to_average : list, optional
        List of variables to include. If None, includes all data variables
    """
    
    print(f"Loading daily dataset: {input_file}")
    ds = xr.open_dataset(input_file)
    
    print(f"Input dataset info:")
    print(f"  Time range: {ds.time.min().values} to {ds.time.max().values}")
    print(f"  Time dimension: {ds.dims['time']} daily timesteps")
    print(f"  Spatial dimensions: {ds.dims['latitude']} lat × {ds.dims['longitude']} lon")
    print(f"  Total data variables: {len(ds.data_vars)}")
    
    # Show data variables
    print(f"\nAvailable variables:")
    for i, var in enumerate(ds.data_vars, 1):
        print(f"  {i:2d}. {var}")
    
    # Determine which variables to process
    if variables_to_average is None:
        # Default: include key SSS-related variables, exclude utility variables
        default_vars = [
            'smap_sss', 'smap_sss_uncertainty', 'anc_sss', 'anc_sst',
            'smap_spd', 'smap_high_spd', 'weight', 'ice_fraction', 'land_fraction'
        ]
        variables_to_average = [var for var in default_vars if var in ds.data_vars]
        
        # If merged dataset has combined variables, include those too
        merged_vars = [var for var in ds.data_vars if any(x in var.lower() for x in ['combined', 'l3', 'l4'])]
        variables_to_average.extend(merged_vars)
        
        # Remove duplicates
        variables_to_average = list(set(variables_to_average))
    
    print(f"\nVariables to be averaged ({len(variables_to_average)}):")
    for var in variables_to_average:
        if var in ds.data_vars:
            coverage = (~ds[var].isnull()).sum().values / ds[var].size * 100
            print(f"  • {var} (data coverage: {coverage:.1f}%)")
    
    # Create monthly grouping
    print(f"\nCreating monthly averages...")
    print("This may take several minutes for large datasets...")
    
    # Group by year-month and calculate means
    monthly_means = ds.groupby('time.month').mean('time', skipna=True)
    
    # Also create year-month grouping for more detailed monthly averages
    print("Creating detailed year-month averages...")
    
    # Add year-month coordinate for grouping
    ds_with_yearmonth = ds.assign_coords(year_month=ds.time.dt.strftime('%Y-%m'))
    monthly_detailed = ds_with_yearmonth.groupby('year_month').mean('time', skipna=True)
    
    # Create a proper time coordinate for the monthly data
    # Use the 1st of each month as representative date
    year_months = pd.to_datetime(monthly_detailed.year_month.values, format='%Y-%m')
    monthly_time_coord = year_months  # 1st of each month (default for pd.to_datetime)
    
    # Replace the year_month coordinate with proper datetime
    monthly_detailed = monthly_detailed.drop_vars('year_month')
    monthly_detailed = monthly_detailed.rename({'year_month': 'time'})
    monthly_detailed = monthly_detailed.assign_coords(time=monthly_time_coord)
    
    # Sort by time
    monthly_detailed = monthly_detailed.sortby('time')
    
    print(f"✅ Monthly averaging complete!")
    print(f"  Original daily data: {ds.dims['time']} timesteps")
    print(f"  Monthly averages: {monthly_detailed.dims['time']} months")
    print(f"  Time range: {monthly_detailed.time.min().values} to {monthly_detailed.time.max().values}")
    
    # Calculate statistics for each variable
    print(f"\nMonthly averaging statistics:")
    for var in variables_to_average:
        if var in ds.data_vars:
            daily_valid = (~ds[var].isnull()).sum().values
            monthly_valid = (~monthly_detailed[var].isnull()).sum().values
            
            print(f"  {var}:")
            print(f"    Daily valid points: {daily_valid:,}")
            print(f"    Monthly valid points: {monthly_valid:,}")
            print(f"    Data retention: {monthly_valid/daily_valid*100:.1f}%")
    
    # Add metadata
    monthly_detailed.attrs = ds.attrs.copy()
    monthly_detailed.attrs['description'] = 'Monthly mean averages of daily SMAP SSS data'
    monthly_detailed.attrs['temporal_resolution'] = 'monthly'
    monthly_detailed.attrs['temporal_averaging'] = 'monthly_mean'
    monthly_detailed.attrs['averaging_method'] = 'arithmetic_mean_skipna'
    monthly_detailed.attrs['source_temporal_resolution'] = 'daily'
    monthly_detailed.attrs['monthly_averaging_date'] = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
    monthly_detailed.attrs['original_time_range'] = f"{ds.time.min().values} to {ds.time.max().values}"
    monthly_detailed.attrs['monthly_time_range'] = f"{monthly_detailed.time.min().values} to {monthly_detailed.time.max().values}"
    monthly_detailed.attrs['original_timesteps'] = ds.dims['time']
    monthly_detailed.attrs['monthly_timesteps'] = monthly_detailed.dims['time']
    
    # Add variable-specific metadata
    for var in monthly_detailed.data_vars:
        if var in ds.data_vars:
            monthly_detailed[var].attrs = ds[var].attrs.copy()
            monthly_detailed[var].attrs['temporal_averaging'] = 'monthly_mean'
            monthly_detailed[var].attrs['averaging_period'] = 'calendar_month'
    
    # Save monthly dataset
    print(f"\nSaving monthly averages to {output_file}...")
    
    # Use compression for efficiency
    encoding = {}
    for var in monthly_detailed.data_vars:
        if monthly_detailed[var].dtype == 'float32':
            encoding[var] = {
                'zlib': True, 
                'complevel': 4,
                'shuffle': True,
                '_FillValue': np.nan
            }
        else:
            encoding[var] = {'zlib': True, 'complevel': 4}
    
    monthly_detailed.to_netcdf(output_file, encoding=encoding)
    
    print(f"✅ Monthly averages saved: {output_file}")
    
    # Quick verification
    print(f"\n=== VERIFICATION ===")
    monthly_check = xr.open_dataset(output_file)
    print(f"Monthly dataset shape: {monthly_check.dims}")
    print(f"File size reduction: {ds.nbytes / monthly_check.nbytes:.1f}x smaller")
    
    # Show sample of time coordinate
    print(f"\nSample monthly time coordinates:")
    for i in range(min(10, len(monthly_check.time))):
        date_str = pd.to_datetime(monthly_check.time[i].values).strftime('%Y-%m-%d')
        print(f"  Month {i+1}: {date_str}")
    
    if len(monthly_check.time) > 10:
        print(f"  ... and {len(monthly_check.time)-10} more months")
    
    monthly_check.close()
    ds.close()
    
    return monthly_detailed

def create_seasonal_means(monthly_file, output_file):
    """
    Create seasonal averages from monthly data
    """
    print(f"\nCreating seasonal averages from {monthly_file}...")
    
    monthly_ds = xr.open_dataset(monthly_file)
    
    # Define seasons
    season_groups = monthly_ds.groupby('time.season').mean('time', skipna=True)
    
    # Create seasonal time coordinates (use middle of season)
    season_dates = {
        'DJF': pd.Timestamp('2020-01-15'),  # Winter
        'MAM': pd.Timestamp('2020-04-15'),  # Spring  
        'JJA': pd.Timestamp('2020-07-15'),  # Summer
        'SON': pd.Timestamp('2020-10-15'),  # Fall
    }
    
    season_coords = [season_dates[season] for season in season_groups.season.values]
    season_groups = season_groups.assign_coords(time=('season', season_coords))
    season_groups = season_groups.swap_dims({'season': 'time'})
    
    # Add metadata
    season_groups.attrs = monthly_ds.attrs.copy()
    season_groups.attrs['temporal_resolution'] = 'seasonal'
    season_groups.attrs['temporal_averaging'] = 'seasonal_mean'
    season_groups.attrs['seasons'] = 'DJF=Winter, MAM=Spring, JJA=Summer, SON=Fall'
    
    # Save seasonal dataset
    encoding = {var: {'zlib': True, 'complevel': 4} for var in season_groups.data_vars}
    season_groups.to_netcdf(output_file, encoding=encoding)
    
    print(f"Seasonal averages saved: {output_file}")
    monthly_ds.close()
    
    return season_groups

if __name__ == "__main__":
    # File paths
    input_file = "SMAP_Daily_subset_combined_OISSSfilled.nc4"
    monthly_output = "SMAP_MO_subset_combined.nc4"
    seasonal_output = "SMAP_Seasonal_subset_combined.nc4"
    
    print("=== CREATING MONTHLY AVERAGES ===")
    monthly_data = create_monthly_means(input_file, monthly_output)
    
    print("\n=== CREATING SEASONAL AVERAGES ===")
    seasonal_data = create_seasonal_means(monthly_output, seasonal_output)
    
    print(f"   1. {monthly_output} (Monthly averages)")
    print(f"   2. {seasonal_output} (Seasonal averages)")
    print(f"\nYou can now use these averaged datasets for analysis!")

=== CREATING MONTHLY AVERAGES ===
Loading daily dataset: SMAP_Daily_subset_combined_OISSSfilled.nc4
Input dataset info:
  Time range: 2015-05-04T00:00:00.000000000 to 2025-06-03T00:00:00.000000000
  Time dimension: 3604 daily timesteps
  Spatial dimensions: 340 lat × 520 lon
  Total data variables: 10

Available variables:
   1. smap_sss_uncertainty
   2. smap_spd
   3. ice_fraction
   4. anc_sst
   5. weight
   6. land_fraction
   7. smap_high_spd
   8. smap_sss
   9. anc_sss
  10. spatial_ref

Variables to be averaged (9):
  • smap_spd (data coverage: 55.6%)
  • smap_sss (data coverage: 55.9%)
  • anc_sss (data coverage: 55.6%)
  • smap_sss_uncertainty (data coverage: 55.6%)
  • weight (data coverage: 99.5%)
  • smap_high_spd (data coverage: 55.6%)
  • land_fraction (data coverage: 55.6%)
  • anc_sst (data coverage: 55.6%)
  • ice_fraction (data coverage: 55.6%)

Creating monthly averages...
This may take several minutes for large datasets...
Creating detailed year-month averages...
